In [1]:
import pandas as pd

# Load data from the parquet file
def load_data():
    return pd.read_parquet('../data/all-data.parquet')
data = pd.read_parquet('../data/all-data.parquet')

In [ ]:

# Generalized aggregation function with dynamic level of aggregation
def aggregate_data(data, aggregation_level, measures=['Sc', 'GrossVP', 'NetVP', 'Stableford']):
    levels = {
        'Pl': ['Pl', 'Player'],
        'TEG': ['Pl', 'TEG', 'Player', 'TEGNum'],
        'Round': ['Pl', 'TEG', 'Round', 'Player', 'TEGNum'],
        'FrontBack': ['Pl', 'TEG', 'Round', 'FrontBack', 'Player', 'TEGNum']
    }
    
    if aggregation_level not in levels:
        raise ValueError(f"Invalid aggregation level: {aggregation_level}. Choose from: {list(levels.keys())}")
    
    group_columns = levels[aggregation_level]
    return data.groupby(group_columns, as_index=False)[measures].sum().sort_values(by=group_columns)

data = pd.read_parquet('../data/all-data.parquet')
#print(data.head())

teg_data = aggregate_data(data,'TEG')
print(teg_data.head())
print(teg_data.shape[0])

rd_data = aggregate_data(data,'Round')
print(rd_data.head())
print(rd_data.shape[0])

nine_data = aggregate_data(data,'FrontBack')
print(nine_data.head())
print(nine_data.shape[0])

In [20]:
import pandas as pd
from utils import aggregate_data, format_vs_par

# Load the data from the Parquet file
all_data = pd.read_parquet('../data/all-data.parquet')

# Filter out TEG 2 and TEG 50 (in place by reassigning to itself)
all_data = all_data[~all_data['TEG'].isin(['TEG 2', 'TEG 50'])]

# Aggregate the data by 'TEG'
teg_data = aggregate_data(all_data, 'TEG')

# Define the fields & number of rows to keep
teg_fields = ['Player', 'TEG', 'GrossVP']
n_keep = 10

# Find the n lowest 'Sc' values and return the corresponding rows
lowest_sc_rows = teg_data[teg_fields].nsmallest(n_keep, 'GrossVP').sort_values(by='GrossVP', ascending=True)
lowest_sc_rows['Rank'] = lowest_sc_rows['GrossVP'].rank(method='min').astype(int).astype(str)
lowest_sc_rows.loc[lowest_sc_rows.duplicated('Rank', keep=False), 'Rank'] += '='
lowest_sc_rows = lowest_sc_rows[['Rank', 'Player', 'TEG', 'GrossVP']]
lowest_sc_rows.rename(columns={'GrossVP': 'Gross'}, inplace=True)
lowest_sc_rows['Gross'] = lowest_sc_rows['Gross'].apply(format_vs_par)

# Print the rows with the lowest 'Sc' values
print(lowest_sc_rows)


def find_lowest_sc_rows(data, level_of_aggregation, fields_to_keep, top_n=10):
    # Aggregate the data based on the provided level of aggregation
    aggregated_data = aggregate_data(data, level_of_aggregation)
    
    # Find the n lowest 'GrossVP' values and return the corresponding rows
    lowest_sc_rows = aggregated_data[fields_to_keep].nsmallest(top_n, 'GrossVP').sort_values(by='GrossVP', ascending=True)
    
    # Add ranking column
    lowest_sc_rows['Rank'] = lowest_sc_rows['GrossVP'].rank(method='min').astype(int).astype(str)
    lowest_sc_rows.loc[lowest_sc_rows.duplicated('Rank', keep=False), 'Rank'] += '='
    
    # Reorder and rename columns
    lowest_sc_rows = lowest_sc_rows[['Rank'] + fields_to_keep]
    lowest_sc_rows.rename(columns={'GrossVP': 'Gross'}, inplace=True)
    
    # Apply formatting to 'Gross' column
    lowest_sc_rows['Gross'] = lowest_sc_rows['Gross'].apply(format_vs_par)
    
    return lowest_sc_rows


rd_fields = ['Player', 'TEG', 'Round', 'GrossVP']
lowest_rounds = find_lowest_sc_rows(all_data,'Round',rd_fields)
print(lowest_rounds)


ImportError: cannot import name 'format_vs_par' from 'utils' (c:\Users\JBA33\OneDrive - Sky\Documents\python\TEG\streamlit\utils.py)

In [21]:
import pandas as pd
from utils import aggregate_data, format_vs_par

# Load the data from the Parquet file & exclude teg 2 and 50
all_data = pd.read_parquet('../data/all-data.parquet')
all_data = all_data[~all_data['TEG'].isin(['TEG 2', 'TEG 50'])]



def find_lowest_sc_rows(data, level_of_aggregation, fields_to_keep, field='GrossVP', top_n=10):
    # Aggregate the data based on the provided level of aggregation
    aggregated_data = aggregate_data(data, level_of_aggregation)
    
    # Define properties for each field
    field_properties = {
        'GrossVP': {'new_name': 'Gross', 'ascending': True, 'formatter': format_vs_par, 'additional_field': 'Sc'},
        'NetVP': {'new_name': 'Net', 'ascending': True, 'formatter': format_vs_par, 'additional_field': None},
        'Sc': {'new_name': 'Gross Score', 'ascending': True, 'formatter': lambda x: int(x), 'additional_field': 'GrossVP'},
        'Stableford': {'new_name': 'Stableford', 'ascending': False, 'formatter': lambda x: int(x), 'additional_field': None},
    }
    
    # Get the properties for the selected field
    properties = field_properties.get(field)
    if not properties:
        raise ValueError(f"Invalid field: {field}")
    
    # Append additional_field to fields_to_keep if it's not None
    additional_field = properties['additional_field']
    print(f"\nField is: {field};\n additional_field is: {additional_field}\nfields to keep: {fields_to_keep}")

    fields_to_keep += [additional_field] if additional_field else []

    print(f"\nfields to keep: {fields_to_keep}\n")
    
    all_fields = fields_to_keep + [field]

    print(f"\nall_fields: {all_fields}")


    # Sort the data based on the 'ascending' property
    sorted_data = (aggregated_data[all_fields]
                   .sort_values(by=field, ascending=properties['ascending'])
                   .head(top_n))

    # Add ranking column (ranking order follows the 'ascending' property)
    sorted_data['Rank'] = sorted_data[field].rank(ascending=properties['ascending'], method='min').astype(int).astype(str)
    sorted_data.loc[sorted_data.duplicated('Rank', keep=False), 'Rank'] += '='
    
    # Reorder and rename columns
    sorted_data = sorted_data[['Rank'] + all_fields]
    sorted_data.rename(columns={field: properties['new_name']}, inplace=True)
    
    # Apply formatting to the chosen field
    sorted_data[properties['new_name']] = sorted_data[properties['new_name']].apply(properties['formatter'])
    
    return sorted_data

n_keep = 10
rd_fields = ['Player', 'TEG', 'Round']

lowest_rounds_gross = find_lowest_sc_rows(all_data,'Round',rd_fields,'GrossVP' ,n_keep)
print('\nBest Gross')
print(lowest_rounds_gross)

print('rd_fields')
print(rd_fields)

lowest_rounds_sc = find_lowest_sc_rows(all_data,'Round',rd_fields,'Sc' ,n_keep)
print('\nBest Score')
print(lowest_rounds_sc)

lowest_rounds_net = find_lowest_sc_rows(all_data,'Round',rd_fields,'NetVP' ,n_keep)
print('\nBest Net')
print(lowest_rounds_net)

best_rounds_stableford = find_lowest_sc_rows(all_data,'Round',rd_fields,'Stableford' ,n_keep)
print('\n=======\nBest Stableford\n========')
print(best_rounds_stableford)

ImportError: cannot import name 'format_vs_par' from 'utils' (c:\Users\JBA33\OneDrive - Sky\Documents\python\TEG\streamlit\utils.py)

In [6]:
df = data
# Print the columns
print("\nColumns in the DataFrame:")
for col in df.columns:
    print(f"- {col}")

# Print the first few rows
print("\nFirst few rows of the DataFrame:")
print(df.head())



Columns in the DataFrame:
- TEG
- Round
- Hole
- PAR
- SI
- Pl
- Sc
- HC
- HCStrokes
- GrossVP
- Net
- NetVP
- Stableford
- TEGNum
- HoleID
- Player
- FrontBack
- Date
- Course
- Hole Order Ever
- Sc Cum Round
- Sc Cum TEG
- Sc Cum Career
- GrossVP Cum Round
- GrossVP Cum TEG
- GrossVP Cum Career
- NetVP Cum Round
- NetVP Cum TEG
- NetVP Cum Career
- Stableford Cum Round
- Stableford Cum TEG
- Stableford Cum Career
- TEG Count
- Career Count
- Sc Round Avg
- Sc TEG Avg
- Sc Career Avg
- GrossVP Round Avg
- GrossVP TEG Avg
- GrossVP Career Avg
- NetVP Round Avg
- NetVP TEG Avg
- NetVP Career Avg
- Stableford Round Avg
- Stableford TEG Avg
- Stableford Career Avg

First few rows of the DataFrame:
     TEG  Round  Hole  PAR  SI  Pl   Sc    HC  HCStrokes  GrossVP  ...  \
0  TEG 7      1     1    5   7  AB  8.0  36.0          2      3.0  ...   
1  TEG 7      1     2    3  13  AB  4.0  36.0          2      1.0  ...   
2  TEG 7      1     3    5  11  AB  6.0  36.0          2      1.0  ...   

In [4]:
import utils
all_data = data
teg_data = utils.aggregate_data(all_data,'TEG')
print(teg_data.head())


   Pl     TEG      Player  TEGNum     Sc  GrossVP  NetVP  Stableford
0  AB  TEG 10  Alex BAKER      10  376.0     90.0  -38.0       184.0
1  AB  TEG 11  Alex BAKER      11  394.0    106.0   10.0       137.0
2  AB  TEG 12  Alex BAKER      12  392.0    104.0    4.0       143.0
3  AB  TEG 13  Alex BAKER      13  390.0    103.0    3.0       141.0
4  AB  TEG 14  Alex BAKER      14  389.0    102.0   -2.0       152.0


In [6]:
def compute_teg_statistics(df):
    """
    Computes the following statistics for each TEG:
    - Sum of GrossVP
    - Sum of Stableford
    
    Identifies:
    - TEG with the lowest sum of GrossVP
    - TEG with the highest sum of Stableford
    - TEG with the lowest sum of Stableford
    
    Parameters:
    df (pd.DataFrame): DataFrame containing TEG data with 'TEG', 'GrossVP', and 'Stableford' columns.
    
    Returns:
    dict: A dictionary containing the identified TEGs and their corresponding sums.
    """
    # Validate required columns
    required_columns = {'TEG', 'GrossVP', 'Stableford'}
    if not required_columns.issubset(df.columns):
        missing = required_columns - set(df.columns)
        raise ValueError(f"Missing columns in DataFrame: {missing}")
    
    # Group by 'TEG' and calculate sum of 'GrossVP' and 'Stableford'
    grouped = df.groupby('TEG').agg({'GrossVP': 'sum', 'Stableford': 'sum'}).reset_index()
    
    # Debug: Show grouped DataFrame
    print("Grouped DataFrame:")
    print(grouped)
    
    # Identify TEG with the lowest sum of GrossVP
    lowest_grossvp_row = grouped.loc[grouped['GrossVP'].idxmin()]
    lowest_grossvp_teg = lowest_grossvp_row['TEG']
    lowest_grossvp_sum = lowest_grossvp_row['GrossVP']
    
    # Identify TEG with the highest sum of Stableford
    highest_stableford_row = grouped.loc[grouped['Stableford'].idxmax()]
    highest_stableford_teg = highest_stableford_row['TEG']
    highest_stableford_sum = highest_stableford_row['Stableford']
    
    # Identify TEG with the lowest sum of Stableford
    lowest_stableford_row = grouped.loc[grouped['Stableford'].idxmin()]
    lowest_stableford_teg = lowest_stableford_row['TEG']
    lowest_stableford_sum = lowest_stableford_row['Stableford']
    
    # Compile results into a dictionary
    results = {
        'Lowest GrossVP': {
            'TEG': lowest_grossvp_teg,
            'GrossVP Sum': lowest_grossvp_sum
        },
        'Highest Stableford': {
            'TEG': highest_stableford_teg,
            'Stableford Sum': highest_stableford_sum
        },
        'Lowest Stableford': {
            'TEG': lowest_stableford_teg,
            'Stableford Sum': lowest_stableford_sum
        }
    }
    
    return results

compute_teg_statistics(teg_data)

Grouped DataFrame:
       TEG  GrossVP  Stableford
0   TEG 10    537.0       932.0
1   TEG 11    471.0       676.0
2   TEG 12    518.0       916.0
3   TEG 13    407.0       727.0
4   TEG 14    321.0       568.0
5   TEG 15    623.0       798.0
6   TEG 16    494.0       699.0
7    TEG 2    444.0       451.0
8    TEG 3    473.0       721.0
9    TEG 4    553.0       622.0
10   TEG 5    460.0       723.0
11  TEG 50    369.0       340.0
12   TEG 6    517.0       684.0
13   TEG 7    504.0      1000.0
14   TEG 8    675.0       754.0
15   TEG 9    599.0       894.0


{'Lowest GrossVP': {'TEG': 'TEG 14', 'GrossVP Sum': 321.0},
 'Highest Stableford': {'TEG': 'TEG 7', 'Stableford Sum': 1000.0},
 'Lowest Stableford': {'TEG': 'TEG 50', 'Stableford Sum': 340.0}}

In [36]:
#COMPUTE BEST & WORST SCORES BY TEG

import pandas as pd

all_data['Year'] = pd.to_datetime(all_data['Date'],format = '%d/%m/%Y').dt.year
# print(all_data['Year'])
teg_yr = all_data[['TEG', 'Year']].drop_duplicates()
teg_yr['Year'] = teg_yr['Year'].fillna(0).astype(int)
#print(teg_yr)


def get_teg_summary(df):
    # Group by 'TEG' and 'Player', and calculate the sum for each player in each TEG
    grouped = df.groupby(['TEGNum','Player']).agg({
        'GrossVP': 'sum',
        'Stableford': 'sum'
    }).sort_values(by="TEGNum").reset_index()

    # Initialize a list to store the results for each TEG
    results = []

    # Get unique TEG values
    for teg in df['TEGNum'].unique():
        # Filter for the current TEG
        teg_data = grouped[grouped['TEGNum'] == teg]
        
        # Get the player with the lowest sum of GrossVP
        lowest_grossvp = teg_data.loc[teg_data['GrossVP'].idxmin()]
        
        # Get the player with the highest sum of Stableford
        highest_stableford = teg_data.loc[teg_data['Stableford'].idxmax()]
        
        # Get the player with the lowest sum of Stableford
        lowest_stableford = teg_data.loc[teg_data['Stableford'].idxmin()]
        
        # Append the result for this TEG
        results.append({
            'TEGNum': teg,
            'TEG': "TEG "+ str(teg),
            'Best Gross': lowest_grossvp['Player'],
            #'Lowest GrossVP Sum': lowest_grossvp['GrossVP'],
            'Best Net': highest_stableford['Player'],
            #'Highest Stableford Sum': highest_stableford['Stableford'],
            'Worst Net': lowest_stableford['Player'],
            #'Lowest Stableford Sum': lowest_stableford['Stableford']
        })
        
    #results_sorted = results.drop(columns=['TEGNum'])
    # Convert results to a DataFrame
    result_df = pd.DataFrame(results).sort_values(by='TEGNum').drop(columns=['TEGNum'])
    #teg_yr['Year'] = teg_yr['Year'].astype(str)
    results_with_year = pd.merge(result_df,teg_yr,on='TEG',how='left')
    results_with_year = results_with_year[['TEG', 'Year', 'Best Net', 'Best Gross', 'Worst Net']]
    
    
    #Replace to correct to history
    results_with_year.loc[results_with_year['TEG'] == 'TEG 5', 'Best Net'] = 'Gregg WILLIAMS'
    results_with_year.loc[results_with_year['TEG'] == 'TEG 5', 'Best Gross'] = 'Stuart NEUMANN'

    return results_with_year

get_teg_summary(teg_data)
# Example usage:
# Assuming df is your DataFrame with the structure you provided
# result = get_teg_summary(df)
# print(result)


,TEG,Year,Best Net,Best Gross,Worst Net
0,TEG 2,2009,David MULLIN,David MULLIN,Henry MELLER
1,TEG 3,2010,Jon BAKER,David MULLIN,Stuart NEUMANN
2,TEG 4,2011,David MULLIN,David MULLIN,Gregg WILLIAMS
3,TEG 5,2012,Gregg WILLIAMS,Stuart NEUMANN,David MULLIN
4,TEG 6,2013,Gregg WILLIAMS,David MULLIN,Henry MELLER
5,TEG 7,2014,Henry MELLER,David MULLIN,Alex BAKER
6,TEG 8,2015,Gregg WILLIAMS,David MULLIN,Stuart NEUMANN
7,TEG 9,2016,John PATTERSON,David MULLIN,Stuart NEUMANN
8,TEG 10,2017,Alex BAKER,David MULLIN,John PATTERSON
9,TEG 11,2018,Jon BAKER,Jon BAKER,David MULLIN


In [11]:
import pandas as pd
from utils import load_all_data, get_teg_winners

all_data = load_all_data()
winners = get_teg_winners(all_data)
#winners.to_clipboard(index=False)
#winners

#REMOVE ASTERISKS
winner_df = winners.replace('*', '', regex=True)


winner_df

# # Melt the DataFrame to have players and competitions in long format
# melted = pd.melt(df, id_vars=['TEG', 'Year'], value_vars=['TEG Trophy', 'Green Jacket', 'HMM Wooden Spoon'],
#                  var_name='Competition', value_name='Player')
# print(melted)

error: nothing to repeat at position 0